This code was taken from a medium article found here
https://bryanf.medium.com/web-scraping-crypto-prices-with-python-41072ea5b5bf
It does not work because coinmarketcap's website has changed

In [34]:
pip install beautifulsoup4

In [35]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import time

In [36]:
cmc = requests.get('https://coinmarketcap.com/')
soup = BeautifulSoup(cmc.content, 'html.parser')

In [37]:
print(soup.title)

#<title class="next-head">Cryptocurrency Market Capitalizations | CoinMarketCaps/title>

<title>Cryptocurrency Prices, Charts And Market Capitalizations | CoinMarketCap</title>


In [38]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, shrink-to-fit=no" name="viewport"/>
  <link crossorigin="anonymous" href="https://s2.coinmarketcap.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://s2.coinmarketcap.com" rel="dns-prefetch"/>
  <link crossorigin="anonymous" href="https://s3.coinmarketcap.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://s3.coinmarketcap.com" rel="dns-prefetch"/>
  <link href="//www.googletagmanager.com" rel="preconnect"/>
  <link as="font" crossorigin="anonymous" href="https://s2.coinmarketcap.com/static/cloud/fonts/cmc-v2/CMC-V2.woff" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="anonymous" href="https://s2.coinmarketcap.com/static/cloud/fonts/inter/Inter-Regular-new.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorig

In [39]:
data = soup.find('script', id="__NEXT_DATA__",type="application/json")

coins = {}

#using data.contents[0] to remove script tags
coin_data = json.loads(data.contents[0])
listings = coin_data['props']['initialState']['cryptocurrency']['listingLatest']['data']

for i in listings:
    coins[str(i['id'])] = i['slug']

In [40]:
for i in coins:
    page = requests.get(f'https://coinmarketcap.com/currencies/{coins[i]}/historical-data/?start=20170701&end=20210628')
    soup = BeautifulSoup(page.content, 'html.parser')
    data = soup.find('script', id="__NEXT_DATA__",type="application/json")
    historical_data = json.loads(data.contents[0])

In [41]:
print(historical_data)

{'props': {'isServer': True, 'initialState': {'app': {'locale': 'en-US', 'theme': 'DAY', 'lang': 'en', 'country': 'us', 'currency': {'id': 2781, 'name': 'United States Dollar', 'symbol': 'usd', 'token': '$'}, 'bottomBannerHeights': {}, 'browser': {}, 'window': {'width': 0, 'height': 0, 'isNarrowLayout': False}, 'modal': {'instance': 0, 'data': {}}, 'message': ''}, 'cryptocurrency': {'listingLatest': {'page': 1, 'sort': '', 'sortDirection': '', 'data': [], 'filters': {}, 'totalItems': 0}, 'listingHistorical': {'data': [], 'page': 1, 'sort': '', 'sortDirection': ''}, 'new': {'page': 1, 'sort': '', 'sortDirection': '', 'data': []}, 'watchlist': {'page': 1, 'sort': '', 'sortDirection': '', 'data': []}, 'map': {'data': [], 'slugMap': {}}, 'info': {}, 'prices': {}, 'quotesLatest': {}, 'quotesHistorical': {'loading': True}, 'ohlcvHistorical': {}, 'related': {'data': {}}, 'marketPairsLatest': {'data': {}}, 'pricePerformanceStatsLatest': {'data': {}}, 'topDerivatives': [], 'yieldFarmingRankingL

In [42]:
quotes = historical_data['props']

In [43]:
market_cap = []
volume = []
timestamp = []
name = []
symbol = []
slug = []

In [44]:
for j in quotes:
    market_cap.append(j['quote']['USD']['market_cap'])
    volume.append(j['quote']['USD']['volume'])
    timestamp.append(j['quote']['USD']['timestamp'])
    name.append(j['name'])
    symbol.append(j['symbol'])
    slug.append(coins[i])


TypeError: string indices must be integers